In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
from tqdm import tqdm_notebook as tqdm
import pickle
from pathlib import Path
from nuscenes import NuScenes
from scipy.spatial.transform import Rotation as R 
from math import cos, sin, pi
from lyft_dataset_sdk.lyftdataset import *
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud, Box, Quaternion
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix
from lyft_dataset_sdk.eval.detection.mAP_evaluation import Box3D, recall_precision

In [6]:
!ls ../../../data

KITTI  lyft  nuscenes


In [8]:
phase = 'train'
data = 'v1.0-trainval' if phase=='train' else 'v1.0-test'
# lyft = LyftDataset(data_path=f'../../../data/lyft/{phase}/', json_path=f'../../../data/lyft/{phase}/{data}/', verbose=0) 
# nusc = NuScenes(dataroot=f'../../data/lyft/{phase}/', version=data, verbose=0) 
nusc = NuScenes(dataroot=f'../../../data/nuscenes/v1.0-trainval/', version=data, verbose=0) 
lyft=nusc

### prepare ground truth json files

Ground truth json files for train/val set are used for evaluation using  lyft's official mAP metric

In [20]:
# info_path = '../../../data/lyft/train/infos_val.pkl'
#info_path = '../../../data/lyft/train/infos_train.pkl'
# info_path = '../../../data/nuscenes/v1.0-trainval/infos_train.pkl'
info_path = '../../../data/nuscenes/v1.0-trainval/infos_val.pkl'
with open(info_path, 'rb') as f:
    info = pickle.load(f)['infos']

In [15]:
# only for nuscenes
NameMapping = {
    'movable_object.barrier': 'barrier',
    'vehicle.bicycle': 'bicycle',
    'vehicle.bus.bendy': 'bus',
    'vehicle.bus.rigid': 'bus',
    'vehicle.car': 'car',
    'vehicle.construction': 'construction_vehicle',
    'vehicle.motorcycle': 'motorcycle',
    'human.pedestrian.adult': 'pedestrian',
    'human.pedestrian.child': 'pedestrian',
    'human.pedestrian.construction_worker': 'pedestrian',
    'human.pedestrian.police_officer': 'pedestrian',
    'movable_object.trafficcone': 'traffic_cone',
    'vehicle.trailer': 'trailer',
    'vehicle.truck': 'truck'
}


In [21]:
gt_data = [] 
for inf in tqdm(info):
    sample_token = inf['token']
    sample = lyft.get('sample', sample_token)
    for ann_token in sample['anns']:
        ann_record = lyft.get('sample_annotation', ann_token)
        try:
            data = {
                'sample_token': sample_token,
                'translation': ann_record['translation'],
                'size': ann_record['size'],
                'rotation': ann_record['rotation'],
    #             'name': ann_record['category_name']
                'name': NameMapping[ann_record['category_name']]
            }
            gt_data.append(data)
        except Exception as e:
            pass # for nuscenes, not using some categories

100%|██████████| 6019/6019 [00:00<00:00, 21968.90it/s]


In [22]:
len(gt_data), gt_data[0]

(187528,
 {'sample_token': 'fd8420396768425eabec9bdddf7e64b6',
  'translation': [242.87, 926.036, 0.898],
  'size': [1.726, 4.257, 1.489],
  'rotation': [0.787419398050721, 0.0, 0.0, -0.616417627565468],
  'name': 'car'})

In [23]:
# gt_data_path = '../../../data/lyft/train/gt_data_val.json'
# gt_data_path = '../../../data/lyft/train/gt_data_train.json'
# gt_data_path = '../../../data/nuscenes/v1.0-trainval/gt_data_train.json'
gt_data_path = '../../../data/nuscenes/v1.0-trainval/gt_data_val.json'
with open(gt_data_path, 'w') as f:
    json.dump(gt_data, f, indent=2)